In [21]:
#%pip install -r requirements.txt
import warnings

# Suppress specific Google Cloud SDK warning
warnings.filterwarnings(
    "ignore",
    message="Your application has authenticated using end user credentials.*",
    category=UserWarning,
    module="google.auth._default"
)
import os
bucket = os.getenv("GCS_BUCKET")

#### Duckdb Integration

In [17]:
import duckdb
from fsspec import filesystem
duckdb.register_filesystem(filesystem('gcs'))

#duckdb read
duckdb.sql(f"select * from read_csv('gs://{bucket}/test.csv')")

┌───────┬───────┬───────┐
│   a   │   b   │   c   │
│ int64 │ int64 │ int64 │
├───────┼───────┼───────┤
│     1 │     2 │     3 │
│     4 │     5 │     6 │
└───────┴───────┴───────┘

In [20]:
#duckdb write
duckdb.sql(f"copy (select * from read_csv('gs://{bucket}/test.csv')) to 'gs://{bucket}/ducks_out.csv'")

#### Polars Integration

In [22]:
import polars as pl

#polars read
df = pl.read_csv(f'gs://{bucket}/test.csv')
df.head()

a,b,c
i64,i64,i64
1,2,3
4,5,6


In [23]:
df.write_csv(f"gs://{bucket}/polars_out.csv")

#### Daft Integration

In [24]:
import daft

#read
df = daft.read_csv(f'gs://{bucket}/test.csv')
df.show()

aInt64,bInt64,cInt64
1,2,3
4,5,6


In [ ]:
#daft write #will create a random guid for the file name...
df.write_csv(f'gs://{bucket}/daft_out')

#### Datafusion Integration

In [26]:
#https://datafusion.apache.org/python/user-guide/data-sources.html#object-store
from datafusion import SessionContext
ctx = SessionContext()

In [27]:
from datafusion.object_store import GoogleCloud
gs = GoogleCloud(bucket_name = bucket)
ctx.register_object_store("gs://", gs, None)
sql = f"""
    CREATE EXTERNAL TABLE test_data (
        a int, b int, c int
    )
    stored as csv
    location 'gs://{bucket}/test.csv'
"""
ctx.sql(sql)
ctx.sql("select * from test_data").show()

DataFrame()
+---+---+---+
| a | b | c |
+---+---+---+
| 1 | 2 | 3 |
| 4 | 5 | 6 |
+---+---+---+


In [28]:
#test wrting
gs_path = f"gs://{bucket}/agg.csv"
ctx.sql(f"copy (select * from test_data) to '{gs_path}'")

2
